<a href="https://colab.research.google.com/github/debalina013/Healthcare-RAG-Chatbot/blob/main/RAG_Internship_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain-community

In [2]:
!pip install pypdf

In [3]:
!pip install fastembed

In [4]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 20.2 MB/s eta 0:00:00


In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import FastEmbedEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA

In [6]:
file_path = '/content/Project/Healthcare1.pdf'

In [7]:
import os

In [8]:
if not os.path.exists(file_path):
    raise FileNotFoundError(f"File not found: {file_path}")

In [9]:
if not file_path.endswith(".pdf"):
    raise ValueError(f"Invalid file type: {file_path}. Expected a PDF file.")

In [10]:
loader = PyPDFLoader(file_path)

In [11]:
data = loader.load()

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

In [13]:
text = text_splitter.split_documents(data)

In [14]:
embeddings = FastEmbedEmbeddings(model_name='BAAI/bge-small-en-v1.5')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

In [15]:
db = FAISS.from_documents(text, embeddings)

In [16]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [17]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 3.0 MB/s eta 0:00:00


In [18]:
!pip install langchain-groq

In [19]:
from groq import Groq

In [20]:
!pip install python-dotenv

In [29]:
with open('.env', 'w') as f:
    f.write('GROQ_API_KEY=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')

In [30]:
from dotenv import load_dotenv

In [31]:
load_dotenv()

True

In [32]:
from langchain_groq import ChatGroq

In [33]:
client = Groq(api_key=os.getenv('GROQ_API_KEY'))

In [34]:
llm = ChatGroq(model_name='llama3-70b-8192')

In [35]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

<ipython-input-35-4cbb8db76ddd>:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [36]:
prompt_template = (
    "You are a helpful assistant. Greet the user and answer the following questions.\n"
    "Chat history:\n{chat_history}\n\n"
    "Context:\n{context}\n\n"
    "Question: {question}\n"
)

In [37]:
prompt = PromptTemplate(template=prompt_template, input_variables=['chat_history', 'context', 'question'])

In [38]:
qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    memory=memory,
    combine_docs_chain_kwargs={'prompt': prompt},
    retriever=retriever
)

In [42]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [43]:
import gradio as gr

In [44]:
def chat_with_bot(user_input, history=[]):
  history = history or []
  result = qa(user_input)
  bot_response = result['answer']
  history.append((user_input, bot_response))
  return history, history

In [50]:
with gr.Blocks() as chatbot_ui:
    gr.Markdown("### 🤖 Chat with Your Healthcare RAG Bot")
    chat_history = gr.Chatbot(label="Chatbot")
    user_input = gr.Textbox(label="Your Message", placeholder="Type your message here...")
    submit_button = gr.Button("Send")
    submit_button.click(chat_with_bot, inputs=[user_input, chat_history], outputs=[chat_history, chat_history])

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:242: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


In [51]:
chatbot_ui.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0b33f8d5d02b80c6d3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
